In [1]:
# import all libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler,OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import math

In [2]:
# load csv
base_df = pd.read_csv('atp_transformed/2000-2024 players_3.csv')

C:\Users\Golbe\AppData\Local\Temp\ipykernel_3944\872059729.py:2: DtypeWarning: Columns (18,41,42) have mixed types. Specify dtype option on import or set low_memory=False.
  base_df = pd.read_csv('atp_transformed/2000-2024 players_3.csv')


In [3]:
full_df = base_df
len(full_df)

1282807

In [4]:
selected_features = [
# 'surface', 
'tourney_level', 
# 'tourney_date', # dropped because not numeric 
'match_num', 
#'player_height', 
# 'player_country', 
'player_age', 
# 'elo_pre_match', # dropped because of colinearity
# 'opponent_elo_pre_match', 
#'opponent_rank', 
# 'mean_numb', # dropped as the rolling means are derrivitives of these columns and adding the raw ones creates a worse model and increases conlinearity
# 'median_numb', 
# 'total_numb', 
# 'mean_diff', 
# 'median_diff', 
# 'total_diff', 
# 'mean_tb_numb', 
# 'median_tb_numb', 
# 'total_tb_numb', 
# 'mean_tb_diff', 
# 'median_tb_diff', 
# 'total_tb_diff', 
'days_of_experience',
# 'career_year', 
# 'rest_days', 
# 'set_dominance', 
# 'tb_dominance', 
#'highest_finish_position', 
'draw_size_rolling_med_10', 
'highest_finish_position_rolling_med_10', 
# 'ace_rolling_mean_10', 
# 'double_faults_rolling_mean_10', # drop these so that most low level players are included
# 'points_on_serve_rolling_mean_10', 
# 'first_serve_in_rolling_mean_10', 
# '1stWon_rolling_mean_10', 
# '2ndWon_rolling_mean_10', 
# 'service_games_rolling_mean_10', 
# 'break_points_saved_rolling_mean_10', 
# 'break_points_faced_rolling_mean_10', 
'elo_pre_match_rolling_mean_10',
'opponent_elo_pre_match_rolling_mean_10', 
'set_dominance_rolling_mean_10', 
'tb_dominance_rolling_mean_10', 
#'player_rank_rolling_mean_10', # dropped because the model will cheat with this if predicting rank
'mean_numb_rolling_mean_10', 
#'median_numb_rolling_mean_10', 
#'total_numb_rolling_mean_10', 
'mean_diff_rolling_mean_10', 
#'median_diff_rolling_mean_10', 
#'total_diff_rolling_mean_10', 
'mean_tb_numb_rolling_mean_10', 
#'median_tb_numb_rolling_mean_10', 
#'total_tb_numb_rolling_mean_10', 
'mean_tb_diff_rolling_mean_10', 
#'median_tb_diff_rolling_mean_10', 
#'total_tb_diff_rolling_mean_10', 
# 'elo_next_match',
'binned_rank' # target
]

In [5]:
full_df[['median_tb_diff','mean_tb_numb','median_tb_numb','mean_tb_diff','tb_dominance']] = full_df[['median_tb_diff','mean_tb_numb','median_tb_numb','mean_tb_diff','tb_dominance']].replace(np.nan,0)

In [6]:
# bin the data using 1-25, 26-100, 101-300, 301-600, 601-1000, 1001-2268 bins as they represent the strength of players in a better way

bins = [1, 25, 100, 300, 600, 1000, 2268]
labels = [0, 1, 2, 3, 4, 5]

full_df['binned_rank'] = pd.cut(
    full_df['filled_player_rank'], 
    bins=bins, 
    labels=labels, 
    include_lowest=True
)

In [7]:
# # drop all nan values
# full_df = full_df.dropna(subset=['player_rank'])

# # log scale the rank
# full_df['log_rank'] = np.log(full_df['player_rank'])

# num_bins = 20

# # Bin the ranks
# full_df['binned_rank'] = pd.qcut(full_df['player_rank'], q=num_bins, labels=False) 

In [8]:
print(len(full_df))
df_subset = full_df[selected_features]
print(len(df_subset))

1282807
1282807


In [9]:
# select all features to use in the prediction
df_subset = full_df[selected_features]

df_subset.isna().sum().sort_values(ascending=False)

mean_tb_diff_rolling_mean_10              112977
mean_tb_numb_rolling_mean_10              112977
tb_dominance_rolling_mean_10              112977
highest_finish_position_rolling_med_10     19971
set_dominance_rolling_mean_10              19342
mean_numb_rolling_mean_10                  19342
mean_diff_rolling_mean_10                  19342
draw_size_rolling_med_10                   19340
elo_pre_match_rolling_mean_10              19275
opponent_elo_pre_match_rolling_mean_10     19275
player_age                                     0
days_of_experience                             0
tourney_level                                  0
match_num                                      0
binned_rank                                    0
dtype: int64

In [10]:
# drop all values where nan because knn cannot deal with empty values
df_subset = df_subset.dropna()

In [11]:
df_subset.isna().sum().sort_values(ascending=False)

tourney_level                             0
match_num                                 0
player_age                                0
days_of_experience                        0
draw_size_rolling_med_10                  0
highest_finish_position_rolling_med_10    0
elo_pre_match_rolling_mean_10             0
opponent_elo_pre_match_rolling_mean_10    0
set_dominance_rolling_mean_10             0
tb_dominance_rolling_mean_10              0
mean_numb_rolling_mean_10                 0
mean_diff_rolling_mean_10                 0
mean_tb_numb_rolling_mean_10              0
mean_tb_diff_rolling_mean_10              0
binned_rank                               0
dtype: int64

From testing the categorical columns add no value to the prediction. Remoiving them creates a better model for all metrics

In [12]:
# # Print non numeric columns
print(len(df_subset))
numeric_cols = df_subset.select_dtypes(exclude=[np.number]).columns
print(numeric_cols)

# encode non numeric values
encoder = OneHotEncoder(sparse_output=False, drop='first')  # drop first to avoid multicollinearity
surface_encoded = encoder.fit_transform(df_subset[[
    # 'surface',
    #'player_country',
    'tourney_level'
    ]])
surface_df = pd.DataFrame(surface_encoded, 
                          columns=encoder.get_feature_names_out([
    #'surface',
    #'player_country',
    'tourney_level'
                                                                 ]),
                          index=df_subset.index)
df_encoded = pd.concat([df_subset.drop([
    #'surface',
    #'player_country',
    'tourney_level'
    ], axis=1), surface_df], axis=1)
#df_encoded = df_subset
print(len(df_encoded))

1169473
Index(['tourney_level', 'binned_rank'], dtype='object')
1169473


In [13]:
# prepare data
X = df_encoded.drop('binned_rank', axis=1)  # Features
y = df_encoded['binned_rank']  # Target

In [14]:
# split into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
# scale features to normalise
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [16]:
# # select K and weights based on library
# from sklearn.model_selection import GridSearchCV
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.pipeline import Pipeline

# pipe = Pipeline([
#     ('scaler', MinMaxScaler()),
#     ('knn', KNeighborsClassifier())
# ])

# param_grid = {
#     'knn__n_neighbors': list(range(1, 40))
# }

# grid = GridSearchCV(pipe, param_grid, cv=5, return_train_score=True)
# grid.fit(X_train, y_train)

# # Extract k values
# k_values = param_grid['knn__n_neighbors']

# # Extract mean test scores and standard deviation from CV
# mean_test_scores = grid.cv_results_['mean_test_score']
# std_test_scores = grid.cv_results_['std_test_score']
# mean_train_scores = grid.cv_results_['mean_train_score']

# plt.figure(figsize=(10,6))
# plt.plot(k_values, mean_train_scores, label='Train Accuracy', marker='o')
# plt.plot(k_values, mean_test_scores, label='CV Accuracy', marker='o')
# plt.fill_between(k_values, mean_test_scores - std_test_scores,
#                  mean_test_scores + std_test_scores, color='gray', alpha=0.2)
# plt.xlabel("Number of Neighbors (k)")
# plt.ylabel("Accuracy")
# plt.title("KNN Train vs CV Accuracy")
# plt.legend()
# plt.grid(True)
# plt.show()

In [17]:
# train
knn = KNeighborsClassifier(n_neighbors=16,weights='uniform')
knn.fit(X_train_scaled, y_train)

,n_neighbors,16
,weights,'uniform'
,algorithm,'auto'
,leaf_size,30
,p,2
,metric,'minkowski'
,metric_params,None
,n_jobs,None


In [18]:
# predict
y_pred = knn.predict(X_test_scaled)

In [19]:
# metrics
precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)
f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)

print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

# Check if there's overfitting
train_score = knn.score(X_train_scaled, y_train)
test_score = knn.score(X_test_scaled, y_test)
print(f"Train accuracy: {train_score:.4f}")
print(f"Test accuracy: {test_score:.4f}")

Accuracy: 0.7159
Precision: 0.7161
Recall: 0.7159
F1 Score: 0.7152


: 

: 

In [ ]:
bin_labels = [f"{bins[i]}-{bins[i+1]}" for i in range(len(bins)-1)]

#bin_edges = pd.qcut(full_df['player_rank'], q=num_bins).cat.categories

# Map numeric predictions to readable strings
#bin_labels = [f"{round(interval.left,2)}-{round(interval.right,2)}" for interval in bin_edges]

y_pred_str = [bin_labels[i] for i in y_pred]
y_test_str = [bin_labels[j] for j in y_test]

In [ ]:
y_pred = pd.Series(y_pred)
results = X_test.reset_index(drop=True).copy()
results['true_rank_coded'] = y_test.reset_index(drop=True)
results['pred_rank_coded'] = y_pred
results['true_rank_coded'] = results['true_rank_coded'].astype(int)
results['pred_rank_coded'] = results['pred_rank_coded'].astype(int)
results['true_rank'] = y_test_str
results['pred_rank'] = y_pred_str
results['correct'] = results['true_rank'] == results['pred_rank']
results['number_of_bins_off'] = results['true_rank_coded'] - results['pred_rank_coded']

In [ ]:
wrong_preds = results[results['correct'] == False]

len(wrong_preds)

In [ ]:
counts = wrong_preds['number_of_bins_off'].value_counts().sort_index()

plt.figure(figsize=(8,5))
plt.bar(counts.index.astype(str), counts.values)
plt.xlabel("Number of Bins Off")
plt.ylabel("Count of Samples")
plt.title("Distribution of Prediction Errors (using bins=20, k=2 and distance weight)")
plt.tight_layout()
plt.show()

In [ ]:
cm = confusion_matrix(y_test, y_pred)

plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()

In [ ]:
wrong_preds